# Issues to Change Requests Metric

### Connect to the Database

In [230]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Repositories in the Database

In [231]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived 
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived
0,chaoss,25155,augur,25440,Parent not available,0.0
1,chaoss,25155,augur-license,25460,Parent not available,0.0
2,chaoss,25155,governance,25449,Parent not available,0.0
3,chaoss,25155,grimoirelab,25448,Parent not available,0.0
4,chaoss,25155,grimoirelab-bestiary,25443,Parent not available,0.0
...,...,...,...,...,...,...
75,SaltStack,60017,salt-winrepo-ng,30356,Parent not available,0.0
76,SaltStack,60017,salty-vagrant,30362,Parent not available,0.0
77,SaltStack,60017,sublime-text,30347,Parent not available,0.0
78,SaltStack,60017,takara,30380,Parent not available,1.0


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
dtype: object

### Repository Filters

In [232]:
# Declare all repo ids you would like to produce charts for.
repo_set = {25440, 25448} # augur, grimoirelab

# graph_dispay can be set as 'repository' or 'competitors'.
# 'repository' will produce a graph of the issues to change requests statistic over time.
# 'competitors' will compare a repository versus other competiting repositories, so it is easy to see where the repository stands.
graph_display = 'repository'

# If graph_display is set to 'competitors', then enter the repository IDs you do no want to alias (compare).
# If graph_display is set to 'repository', then this list will not effect anything.
not_aliased_repos = [25440, 25448]

# group_by can be set as 'month' or 'year'. This determines the span of time for graphs when graph_display is set to 'repository'.
group_by = 'month'

# Specify the dates for filtering.
# If the 'end_date' is in the future, then the 'end_date' will default to the current date.
start_date = '2021-01-01'
end_date = '2021-12-31'

if end_date > pd.to_datetime('today').strftime("%Y-%m-%d"):
    end_date = pd.to_datetime('today').strftime("%Y-%m-%d")

### Query Issues and Change Requests Data

In [233]:
issues_df = pd.DataFrame() # df stands for 'Data Frame'
pullrequests_df = pd.DataFrame()

for repo_id in repo_set:
    issues_query = salc.sql.text(f"""
    SELECT DISTINCT repo_id, COUNT(issue_id) AS issues_count
    FROM augur_data.issues
    WHERE repo_id = {repo_id}
    GROUP BY repo_id
    ORDER BY repo_id
""")
    df_first_repo = pd.read_sql(issues_query, con=engine)
    if not issues_df.empty:
        issues_df = pd.concat([issues_df, df_first_repo]) 
    else:
        issues_df = df_first_repo # first repo
    
    pullrequests_query = salc.sql.text(f"""
    SELECT DISTINCT repo_id, COUNT(pull_request_id) AS pr_count
    FROM augur_data.pull_requests 
    WHERE repo_id = {repo_id}
    GROUP BY repo_id
    ORDER BY repo_id
""")
    prdf_first_repo = pd.read_sql(pullrequests_query, con=engine)
    if not pullrequests_df.empty:
        pullrequests_df = pd.concat([pullrequests_df, prdf_first_repo])
    else:
        pullrequests_df = prdf_first_repo

issues_df.index = [x for x in range(0, len(issues_df.values))]
pullrequests_df.index = [x for x in range(0, len(issues_df.values))]

months_df = pd.DataFrame() # Months Data Frame

months_query = salc.sql.text(f"""
    SELECT * FROM (
        SELECT
            date_part( 'year', created_month :: DATE ) AS year,
            date_part( 'month', created_month :: DATE ) AS MONTH
        FROM (
            SELECT * FROM ( 
                SELECT created_month :: DATE FROM generate_series (TIMESTAMP '{start_date}', TIMESTAMP '{end_date}', INTERVAL '1 month' ) created_month ) d ) x 
        ) y
""")
months_df = pd.read_sql(months_query, con=engine)

print("Issues Data Frame")
display(issues_df)
print("Pull Requests Data Frame")
display(pullrequests_df)
print("Months Data Frame")
display(months_df)


Issues Data Frame


,repo_id,issues_count
0,25440,1028
1,25448,398


Pull Requests Data Frame


,repo_id,pr_count
0,25440,889
1,25448,177


Months Data Frame


,year,month
0,2021.0,1.0
1,2021.0,2.0
2,2021.0,3.0
3,2021.0,4.0
4,2021.0,5.0


### Calculate the Ratio of Issues to Change Requests per Repository

In [234]:
ratios = []
for i in range(0, len(repo_set)):
    ratios.append(issues_df.loc[i, 'issues_count']/pullrequests_df.loc[i, 'pr_count']) # loc format: row, col
    print("Repository ID {}: ".format(issues_df.loc[i, 'repo_id']))
    print("\tIssues: {}\tPull Requests: {}".format(issues_df.loc[i, 'issues_count'], pullrequests_df.loc[i, 'pr_count']))
    print("\tRatio: {}".format(ratios[i]))

# TO DO FROM HERE: CREATE VISUALIZATION

Repository ID 25440: 
	Issues: 1028	Pull Requests: 889
	Ratio: 1.156355455568054
Repository ID 25448: 
	Issues: 398	Pull Requests: 177
	Ratio: 2.248587570621469


####Create Visualizations

In [235]:
from bokeh.plotting import figure, show

p=figure(title="Ratio of Issues to Change Requests", x_axis_label='Month', y_axis_label='Issues')

#test
x=[1,2,3,4,5,6]

y=[]
for i in range(0, 5):
    y.append(months_df.loc[i, 'month'])

p.vbar(x=x, top=y, legend_label="Issues", width=0.5, bottom=0, color="blue")

show(p)

In [236]:
from bokeh.plotting import figure, show

x=[]
y1=[]
y2=[]

for i in range(0, 2):
    x.append(i)
    y1.append(issues_df.loc[i, 'issues_count'])
    y2.append(pullrequests_df.loc[i, 'pr_count'])
    
stacked_bar_df = pd.DataFrame({'x': x, 'y1':y1, 'y2':y2})

q=figure(title="Issues and Change Requests", x_axis_label='Repository', y_axis_label='Number')

q.vbar_stack(['y1', 'y2'], x='x', legend_label=("Issues","Change Requests"), color=("blue", "red"), width=0.5, source=stacked_bar_df)
    


show(q)